# Refactor Public Goods Game


## Overall TODOS

In [ ]:
# Added specified player type proportions (population wide / or within groups if I am staying with group_size == 4 num_membership == 2)
# Completed 

#TODO Implement improved player-types (altruist and freerider) for split endowment case (make them a little more responsive versus totally static and identical) 
# aka varying degrees of altruism : debating this one – for behavioral / lab results reasons 

#DID Add player types and improved decision rule(s) for shared endowment case 
# Started on. The current decison rule works. Want to improve it even more
# M-20 Improved on it even more. Added between-group comparison for SHARED endowment.

#TODO
# I now want to change the type of randomness used. Rather than a uniform distribution, maybe something with a tail, etc. 

#TODO Behavior Issue. Check current output. Currently, very drastic changes in the first round, but then almost no changes occur later on. This should be worked on.

#TODO Additional issue : I do not think my scale factor is not working properly

#DID Initialize contributions for SHARED endowment case with player types
# Completed

#DID Fix Simulate Game Function for Shared Endowment Case
# Completed

#DID Improve the way my csv file looks and make sure it works with the monte carlo simulation portion (I think I am only saving the final iterations data as it stands)
# Completed. It now saves data from all iterations of the Simulation. HOWEVER !!!

## Completed Switch from numpyarray for MOI ==> Pandas DF. to_csv, as well as naming the rows/columns will be of
# great importance for my capacity to use the data in the future 
# Did, but now I cannot open the CSV FILE PROPERLY

#TODO Once I do the above, I can make better graphs that show (1) average group level cooperation, (2) average player level coop, (3) average round level coop, etc.

#TODO design in a way to allow for specific comparissons between the nature of the endowment (SHARED == 0/1) with same parameter sets 

#TODO parameterize to explore the parameter space

#TODO implement power analysis by estimating effect size given the information that I am storing 

#TODO I would be interested in creating some visual component for displaying the change of cooperation of time given different parameter sets -- potentially with
# a scroller feature 

# Import Utils

In [8]:
from src.public_goods_utils import *
import matplotlib.pyplot as plt

In [ ]:
groups_example = formed_groups_matrix(GROUP_SIZE, NUM_GROUPS)
np.argwhere(groups_example == 1) # GOOD FOR VECTORIZING LATER; gives me indices of what groups ppl are in. Will no longer need to use for loop

### Play Game

In [9]:
def simulate_game_new(NUM_MEMBERSHIP, ENDOWMENT, SHARED, GROUP_SIZE, R, NUM_GROUPS, NUM_PLAYERS, NUM_ROUNDS, METHOD):
    """
    Simulate multiple rounds of the multi-group public goods game. The different cases are for which endowment regime is in place, SHARED or SPLIT
    for which there exists two distinct function families
    """
    player_types_matrix = initialize_player_types(NUM_PLAYERS, update_contributions_split_dict, METHOD)
    contributions_matrix = initailize_contribtions_with_player_types(NUM_PLAYERS, NUM_MEMBERSHIP, ENDOWMENT, SHARED, player_types_matrix)
    groups_matrix = formed_groups_matrix(GROUP_SIZE, NUM_GROUPS)

    metrics_of_interest_array = metrics_of_interest_array_builder(NUM_GROUPS, GROUP_SIZE)
    metrics_of_interest_array = update_metrics_of_interest(metrics_of_interest_array, contributions_matrix)
    if SIMS == 1:
        np.savetxt('moi.csv', metrics_of_interest_array, delimiter=',') 
    with open('moi.csv', 'a',  newline = '\n') as f:
        f.write(f"Round Number INIT\n")
        np.savetxt(f, metrics_of_interest_array, delimiter=',')

    round_avg_contribution = np.zeros(NUM_ROUNDS+1) # keeping for now. Stores the round level average contribution across all groups & players
    round_avg_contribution[0] = np.sum(contributions_matrix) / (NUM_MEMBERSHIP*NUM_PLAYERS) # the bottom right most cell of m_o_i_a stores this also

    if SHARED == 1:
        for round in range(NUM_ROUNDS):
            scaled_public_good_matrix = calculuate_scaled_public_good(contributions_matrix, groups_matrix, NUM_PLAYERS, R)
            payoff_matrix = calculate_payoff_shared(contributions_matrix, groups_matrix, ENDOWMENT, scaled_public_good_matrix)
            contributions_matrix = update_contributions_shared(player_types_matrix, contributions_matrix, payoff_matrix, groups_matrix, ENDOWMENT, NUM_MEMBERSHIP)

            round_avg_contribution[round+1] = np.sum(contributions_matrix) / (NUM_MEMBERSHIP*NUM_PLAYERS)

            metrics_of_interest_array = metrics_of_interest_array_builder(NUM_GROUPS, GROUP_SIZE)
            metrics_of_interest_array = update_metrics_of_interest(metrics_of_interest_array, contributions_matrix)
            with open('moi.csv', 'a',  newline = '\n') as f:
                f.write(f"Round Number {round}\n")
                np.savetxt(f, metrics_of_interest_array, delimiter=',')

    else:
        for round in range(NUM_ROUNDS):
            scaled_public_good_matrix = calculuate_scaled_public_good(contributions_matrix, groups_matrix, NUM_PLAYERS, R)
            payoff_matrix = calculate_payoff_split(contributions_matrix, groups_matrix, ENDOWMENT, scaled_public_good_matrix)
            contributions_matrix = update_contributions_split(player_types_matrix, contributions_matrix, payoff_matrix, groups_matrix, ENDOWMENT, NUM_MEMBERSHIP)
            
            round_avg_contribution[round+1] = np.sum(contributions_matrix) / (NUM_MEMBERSHIP*NUM_PLAYERS)

            metrics_of_interest_array = metrics_of_interest_array_builder(NUM_GROUPS, GROUP_SIZE)
            metrics_of_interest_array = update_metrics_of_interest(metrics_of_interest_array, contributions_matrix)
            with open('moi.csv', 'a', newline= '\n') as f:
                f.write(f"Round Number {round}\n")
                np.savetxt(f, metrics_of_interest_array, delimiter=',')
                
    return  round_avg_contribution

In [11]:
game = simulate_game_new(NUM_MEMBERSHIP, ENDOWMENT, SHARED, GROUP_SIZE, R, NUM_GROUPS, NUM_PLAYERS, NUM_ROUNDS, METHOD)
game

array([19.90625, 12.4375 , 11.28125, 13.375  , 11.5625 , 12.9375 ,
       10.6875 , 11.6875 , 11.75   ,  9.5625 , 11.09375])

In [ ]:
pd.read_csv("moi.csv")

## Monte Carlo Simulation & Graphs 

In [ ]:
#DID I think I will always be saving over my csv and replacing the previous iterations data with the way it is currently set up in simulate_game once
# I enter the monte carlo stage
# FIXED

# HOWEVER

# DID I want to make my DF more readible and I think the best way to do this is to switch the current MEtrics of Interest Function from being a NUMPY ARRAy =>
# being a PANDAS DF. Which will give me access to functions found in pandas and row/column names. I will look into making this transition. 
# TODO
# March 21st – Compelted, but Now I cannot open the csv file. It lags my computer. Will need to resolve this. For now loooking at Round average contr is fine


In [ ]:
def monte_carlo_simulation(SIMS):
  """
  Run game with same parameters simulation number of times. Stores as a data frame currently
  """
  current_avg_contribution = simulate_game_new(NUM_MEMBERSHIP, ENDOWMENT, SHARED, GROUP_SIZE, R, NUM_GROUPS, NUM_PLAYERS, NUM_ROUNDS, METHOD)
  avg_contribution_df = pd.DataFrame(current_avg_contribution).T
  expected_round_contributions = np.zeros(NUM_ROUNDS+1)

  base = np.zeros([NUM_PLAYERS + 1, NUM_GROUPS + 1])
  np.savetxt('moi.csv', base, delimiter=',') 

  for sim in range(SIMS-1):
    current_avg_contribution = simulate_game_new(NUM_MEMBERSHIP, ENDOWMENT, SHARED, GROUP_SIZE, R, NUM_GROUPS, NUM_PLAYERS, NUM_ROUNDS, METHOD)
    current_avg_contribution_df = pd.DataFrame(current_avg_contribution).T
    avg_contribution_df = avg_contribution_df.append(current_avg_contribution_df, ignore_index = True)
  cols = avg_contribution_df.columns[:]
  avg_contribution_df[11] = avg_contribution_df[cols].sum(axis=1) / NUM_ROUNDS
  for round in range(NUM_ROUNDS+1):
    expected_round_contributions[round] = avg_contribution_df[round].sum() / SIMS
  return avg_contribution_df, expected_round_contributions

In [ ]:
SHARED = 1
example_MC = monte_carlo_simulation(SIMS)
example_MC[1] # Below results are for when SHARED == 1 

In [ ]:
example_MC_2 = monte_carlo_simulation(SIMS)
example_MC_2[1] # Below results have same parameters but SHARED == 0 

In [ ]:
x = [i for i in range(NUM_ROUNDS)]
y = example_MC[1][:-1]
plt.title(f"Endowment: {ENDOWMENT}, Shared: {SHARED}, In How many groups?: {NUM_MEMBERSHIP}, Group Size: {GROUP_SIZE}, Sims: {SIMS}")
plt.xlabel("Round")
plt.ylabel("Expected Round Contribution")
plt.plot(x, y, color ="green")
plt.show()

In [ ]:
x1 = [i for i in range(NUM_ROUNDS)]
y1 = example_MC_2[1][:-1]
plt.title(f"Endowment: {ENDOWMENT}, Shared: {SHARED}, In How many groups?: {NUM_MEMBERSHIP}, Group Size: {GROUP_SIZE}, Sims: {SIMS}")
plt.xlabel("Round")
plt.ylabel("Expected Round Contribution")
plt.plot(x1, y1, color ="green")
plt.show()